In [1]:
#
import sys
sys.path.append('./')

In [18]:
# General imports
import json
from qiskit import QuantumCircuit
import numpy as np
from dataclasses import dataclass
from enum import auto, Enum

In [23]:
@dataclass
class SchedulerType(Enum):
    """The type of scheduler to use."""

    BASELINE = auto()
    SIMPLE = auto()
    EXTENDED = auto()  

In [24]:
# Configuration
# 1. Number of Jobs in the batch
jobs = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5}
# 2. Number of machines
machines = {"QUITO": 5, "BELEM": 5}
# 3. Scheduling type
name_schedule = SchedulerType.EXTENDED


In [25]:
# Setup
if name_schedule == SchedulerType.EXTENDED:
    from src.algorithm import MILQ
    bigM = 1000
    timesteps = 2**6

In [26]:
# Scheduler
if name_schedule == "MILQ":
    MILQ.example_problem(bigM, timesteps, jobs, machines, "./results/MILQ")


In [ ]:
# Visualize





In [ ]:

# Analyze



